# Creating the Train dataset

**Using the free LLaMA 3-70B model from Groq and Pydantic to create clear and consistent training data.**

## Import dataset with job offers

In [ ]:
import pandas as pd

In [ ]:
df = pd.read_csv('/content/combined_jobs.csv')

In [ ]:
df.shape

(1079, 6)

In [ ]:
df.head()

Position                Company_Name  \
0                                IT Manager  10 Percent Recruiting Ltd.   
1                       Manager, IT Support                      Procom   
2        Director of Information Technology   Southampton Financial Inc   
3  Manager, Information Technology Services         Town of Tillsonburg   
4                           Systems Manager                       Accor   

                              Location Post_Month  Post_Year  \
0  Vancouver, British Columbia, Canada       June       2024   
1             Toronto, Ontario, Canada       June       2024   
2             Toronto, Ontario, Canada       June       2024   
3         Tillsonburg, Ontario, Canada       June       2024   
4           Winnipeg, Manitoba, Canada       June       2024   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                               

In [ ]:
df = df.head(500)

## Generative AI processing

In [ ]:
# We will generate a new column that contains the following JSON object for each job Description in the following dataset

"""
{
  "role_summary": "A concise, non-technical summary of the job role. It should describe the primary responsibilities of the role in simple language, avoiding jargon. Focus on what the role does rather than listing requirements.",
  "key_terms": [
    {
      "term": "Technical Term or jargon from the job description",
      "explanation": "A simple explanation of the term in the context of the role. This helps recruiters understand technical jargon without needing domain expertise."
    }
  ],
  "skill_priorities": {
    "must_have": ["List of essential skills required for the role. These are non-negotiable and should be explicitly mentioned in the job description."],
    "nice_to_have": ["List of preferred skills that are beneficial but not mandatory. These are often marked as 'preferred,' 'a plus,' or 'optional' in the job description."]
  },
  "proposed_screening_questions_with_answers": [
    {
      "question": "A role-specific question to assess candidate expertise. The question should focus on technical skills, problem-solving, or past experiences relevant to the role.",
      "example_answer": "An example of a strong candidate response to the question. The answer should demonstrate technical depth, problem-solving, and relevance to the role."
    }
  ],
  "red_flags": [
    "Indicators of potential mismatches for the role, offering actionable insights for recruiters based on the job offer. Examples include 'Avoid candidates without cloud experience'."
  ],
  "confidence_score": "A numerical score between 0 and 100 indicating the model's confidence in the accuracy of its analysis."
}
"""

'\n{\n  "role_summary": "A concise, non-technical summary of the job role. It should describe the primary responsibilities of the role in simple language, avoiding jargon. Focus on what the role does rather than listing requirements.",\n  "key_terms": [\n    {\n      "term": "Technical Term or jargon from the job description",\n      "explanation": "A simple explanation of the term in the context of the role. This helps recruiters understand technical jargon without needing domain expertise."\n    }\n  ],\n  "skill_priorities": {\n    "must_have": ["List of essential skills required for the role. These are non-negotiable and should be explicitly mentioned in the job description."],\n    "nice_to_have": ["List of preferred skills that are beneficial but not mandatory. These are often marked as \'preferred,\' \'a plus,\' or \'optional\' in the job description."]\n  },\n  "proposed_screening_questions_with_answers": [\n    {\n      "question": "A role-specific question to assess candidate

In [ ]:
system_message_prompt = """
You are an AI assistant that converts job descriptions into structured JSON data to assist recruiters. Your task is to extract key details from a given job description and format them into a JSON object with the following structure:

{
  "role_summary": "A concise, non-technical summary of the job role. It should describe the primary responsibilities of the role in simple language, avoiding jargon. Focus on what the role does rather than listing requirements.",
  "key_terms": [
    {
      "term": "Technical Term or jargon from the job description",
      "explanation": "A simple explanation of the term in the context of the role. This helps recruiters understand technical jargon without needing domain expertise."
    }
  ],
  "skill_priorities": {
    "must_have": ["List of essential skills required for the role. These are non-negotiable and should be explicitly mentioned in the job description."],
    "nice_to_have": ["List of preferred skills that are beneficial but not mandatory. These are often marked as 'preferred,' 'a plus,' or 'optional' in the job description."]
  },
  "proposed_screening_questions_with_answers": [
    {
      "question": "A role-specific question to assess candidate expertise. The question should focus on technical skills, problem-solving, or past experiences relevant to the role.",
      "example_answer": "An example of a strong candidate response to the question. The answer should demonstrate technical depth, problem-solving, and relevance to the role."
    }
  ],
  "red_flags": [
    "Indicators of potential mismatches for the role, offering actionable insights for recruiters based on the job offer. Examples include 'Avoid candidates without cloud experience'."
  ],
  "confidence_score": "A numerical score between 0 and 100 indicating the model's confidence in the accuracy of its analysis."
}

Guidelines:
- Ensure the role summary is written in simple, non-technical language, focusing on primary responsibilities rather than qualifications.
- Extract key technical terms and provide clear, non-technical explanations.
- Identify and categorize skills into 'must-have' and 'nice-to-have' based on explicit mentions in the job description.
- Generate screening questions that assess relevant competencies and provide an example of a strong answer.
- Highlight red flags that indicate potential mismatches, ensuring they are actionable.
- Assign a confidence score (0-100) reflecting the reliability of the extracted insights.

Output only the JSON object without additional text.

"""

In [ ]:
!pip install groq

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.9/124.9 kB 6.5 MB/s eta 0:00:00


In [ ]:
import re
import groq
import os
import json
import time
from typing import List, Optional
import json
from pydantic import BaseModel
from google.colab import userdata
from pprint import pprint

In [ ]:
llama3_model= 'llama3-70b-8192'

In [ ]:
key_1 = userdata.get('ahmed92_groq_key')
key_2 = userdata.get('hamza_groq')
key_3 = userdata.get('tunDC_groq')

api_keys = [key_1, key_2, key_3]

In [ ]:
from typing import List, Optional
from pydantic import BaseModel

class KeyTerm(BaseModel):
    term: str
    explanation: str

class SkillPriorities(BaseModel):
    must_have: List[str]
    nice_to_have: List[str]

class ScreeningQuestion(BaseModel):
    question: str
    example_answer: str

class JobAnalysis(BaseModel):
    role_summary: str
    key_terms: List[KeyTerm]
    skill_priorities: SkillPriorities
    proposed_screening_questions_with_answers: List[ScreeningQuestion]
    red_flags: List[str]
    confidence_score: float

In [ ]:
from pydantic import ValidationError
from groq import Groq, APIError, APIConnectionError, RateLimitError
from pydantic import ValidationError
from typing import Union

def analyze_job_with_retries(
    job_description: str,
    prompt: str,
    model_name: str,
    api_keys: list,
    max_retries: int = 3,
    initial_retry_delay: float = 1.0
) -> Union[JobAnalysis, str]:  # Updated return type
    """
    Analyzes a job description with API key rotation on exceptions.

    Args:
        job_description: The job description text to analyze.
        prompt: System prompt for the AI model.
        model_name: Name of the LLM model to use.
        api_keys: List of API keys to rotate through.
        max_retries: Maximum number of retry attempts across all keys.
        initial_retry_delay: Initial delay between retries in seconds (exponential backoff).

    Returns:
        Union[JobAnalysis, str]: Parsed job analysis results or 'Error' if all retries fail.
    """
    retry_delay = initial_retry_delay
    total_attempts = 0

    while total_attempts < max_retries:
        for i, api_key in enumerate(api_keys):
            client = Groq(api_key=api_key)
            try:
                chat_completion = client.chat.completions.create(
                    messages=[
                        {
                            "role": "system",
                            "content": f"{prompt}\n"
                                       f"The JSON object must use the schema: {json.dumps(JobAnalysis.model_json_schema(), indent=2)}",
                        },
                        {
                            "role": "user",
                            "content": f"Analyze this job description:\n\n{job_description}",
                        },
                    ],
                    model=model_name,
                    temperature=0,
                    response_format={"type": "json_object"},
                )
                print(f"Success from {model_name} using API key {i + 1}")
                job_object = JobAnalysis.model_validate_json(
                    chat_completion.choices[0].message.content
                )

                return job_object.model_dump_json()


            except (APIError, APIConnectionError, RateLimitError) as e:
                print(f"API error with key {i + 1}: {str(e)}. Rotating to next key...")
                total_attempts += 1
                if total_attempts >= max_retries:
                    break  # Exit if max retries reached
                time.sleep(retry_delay)
                retry_delay *= 2  # Exponential backoff

            except Exception as e:
                print(f"Unexpected error: {str(e)}. Rotating to next key...")
                total_attempts += 1
                if total_attempts >= max_retries:
                    break  # Exit if max retries reached
                time.sleep(retry_delay)
                retry_delay *= 2

    print("All API keys and retries exhausted. Returning error response.")
    return 'Error'  # Return string 'Error' if all retries fail

In [ ]:
# Example usage
job_desc = """
Note: By applying to this position you will have an opportunity to share your preferred working location from the following: Cambridge, MA, USA; Atlanta, GA, USA; Austin, TX, USA; Seattle, WA, USA.Minimum qualifications:

Master's degree in a quantitative discipline such as Statistics, Engineering, Sciences, or equivalent practical experience
4 years of experience using analytics to solve product or business problems, coding (e.g., Python, R, SQL), querying databases or statistical analysis.
4 years of experience in business visualization tools like Looker, Tableau, etc.

Preferred qualifications:

Master's degree in Computer Science, Economics, or Mathematics, or a related field.
5 years of work experience in data science or quantitative analytics with focus on statistical modeling, Machine Learning, and AI.
Experience with both SQL and Python.
Experience with Machine Learning, AI or AI Pipeline.
Experience in Google Cloud Platform.

About The Job

Cloud Learning Services (CLS) is revolutionizing direct cloud learning. We empower users of all levels with interactive labs and guided experiences to build practical skills on Google Cloud Platform and other leading technologies. Our mission is to make the cloud accessible, engaging, and enjoyable to learn.

As a Business Data Scientist, you will play a key role in uncovering valuable insights from diverse data sources to solve critical business tests. You will address a wide range of exciting projects, from establishing new measurement frameworks to identifying meaningful patterns in large datasets, ultimately empowering stakeholders to make data-driven decisions.

In this role, you will need to be a detail-oriented problem-solver with a strong foundation in data analytics, data visualization, and Artificial Intelligence/Machine Learning. You will solve the real-world problems and a commitment to continuous learning. Exceptional communication and stakeholder management skills are essential, as you will collaborate extensively with cross-functional teams. If you succeed in dynamic environments and are eager to make a real impact with data, we encourage you to apply!

The US base salary range for this full-time position is $166,000-$244,000 + bonus + equity + benefits. Our salary ranges are determined by role, level, and location. Within the range, individual pay is determined by work location and additional factors, including job-related skills, experience, and relevant education or training. Your recruiter can share more about the specific salary range for your preferred location during the hiring process.

Please note that the compensation details listed in US role postings reflect the base salary only, and do not include bonus, equity, or benefits. Learn more about benefits at Google .

Responsibilities

Extract actionable insights from large, complex datasets and build data products like dashboards to operationalize them, driving measurable improvements in Key Performance Indicators (KPIs).
Present and communicate actionable insights and recommendations to executives, leaders, and cross-functional partners, including Product, Engineering, and Marketing teams.
Serve as a peer-reviewer and consultant to other members of the team, fostering a collaborative and knowledge-sharing environment.
Learn and share knowledge of the latest advancements in AI/ML and data science that are relevant to our work.

"""

analysis = analyze_job_with_retries(job_desc, system_message_prompt, llama3_model, api_keys)


# Assuming `analysis` is your JobAnalysis object
pprint(analysis)


Success from llama3-70b-8192 using API key 1
('{"role_summary":"As a Business Data Scientist, you will uncover valuable '
 'insights from diverse data sources to solve critical business tests, '
 'empowering stakeholders to make data-driven '
 'decisions.","key_terms":[{"term":"Machine Learning","explanation":"A type of '
 'Artificial Intelligence that enables computers to learn from data and make '
 'predictions or decisions without being explicitly programmed."},{"term":"AI '
 'Pipeline","explanation":"A series of processes that enable the development, '
 'deployment, and management of Artificial Intelligence '
 'models."},{"term":"Google Cloud Platform","explanation":"A suite of cloud '
 'computing services offered by Google, including data storage, analytics, and '
 'machine learning capabilities."},{"term":"Data '
 'Visualization","explanation":"The process of creating graphical '
 'representations of data to communicate insights and trends more '
 'effectively."},{"term":"SQL","e

In [ ]:
from tqdm.notebook import tqdm
import time

def process_dataframe(df, system_message_prompt, llama3_model, api_keys):
    tqdm.pandas(desc="Processing job descriptions")

    def safe_analyze(row):
        job_desc = row['Details']
        position = row['Position']
        print(f"Processing position: {position}")

        # Analyze the job description
        result = analyze_job_with_retries(job_desc, system_message_prompt, llama3_model, api_keys)

        # Handle the case where the result is 'Error'
        if result == 'Error':
            print(f"Failed to analyze position: {position}")
            return None  # Or return a default value or error object
        else:
            return result  # Return the JobAnalysis object

    # Use progress_apply to process each row
    df['parsed_description'] = df.progress_apply(safe_analyze, axis=1)
    return df

# Process the DataFrame
analysed_df = process_dataframe(df, system_message_prompt, llama3_model, api_keys)

In [ ]:
# analysed_df.to_csv('combined_jobs_with_analysis.csv', index=False)

In [ ]:
# from google.colab import files
# files.download('combined_jobs_with_analysis.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
!pip install gradio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.2/46.2 MB 20.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.2/322.2 kB 23.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.9/94.9 kB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.3/11.3 MB 80.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 kB 5.8 MB/s eta 0:00:00


In [ ]:
system_prompt = """
You are an AI assistant that converts job descriptions into structured JSON data to assist recruiters. Your task is to extract key details from a given job description and format them into a JSON object with the following structure:

{
  "role_summary": "A concise, non-technical summary of the job role. It should describe the primary responsibilities of the role in simple language, avoiding jargon. Focus on what the role does rather than listing requirements.",
  "key_terms": [
    {
      "term": "Technical Term or jargon from the job description",
      "explanation": "A simple explanation of the term in the context of the role. This helps recruiters understand technical jargon without needing domain expertise."
    }
  ],
  "skill_priorities": {
    "must_have": ["List of essential skills required for the role. These are non-negotiable and should be explicitly mentioned in the job description."],
    "nice_to_have": ["List of preferred skills that are beneficial but not mandatory. These are often marked as 'preferred,' 'a plus,' or 'optional' in the job description."]
  },
  "proposed_screening_questions_with_answers": [
    {
      "question": "A role-specific question to assess candidate expertise. The question should focus on technical skills, problem-solving, or past experiences relevant to the role.",
      "example_answer": "An example of a strong candidate response to the question. The answer should demonstrate technical depth, problem-solving, and relevance to the role."
    }
  ],
  "red_flags": [
    "Indicators of potential mismatches for the role, offering actionable insights for recruiters based on the job offer. Examples include 'Avoid candidates without cloud experience'."
  ],
  "confidence_score": "A numerical score between 0 and 100 indicating the model's confidence in the accuracy of its analysis."
}

Guidelines:
- Ensure the role summary is written in simple, non-technical language, focusing on primary responsibilities rather than qualifications.
- Extract key technical terms and provide clear, non-technical explanations.
- Identify and categorize skills into 'must-have' and 'nice-to-have' based on explicit mentions in the job description.
- Generate screening questions that assess relevant competencies and provide an example of a strong answer.
- Highlight red flags that indicate potential mismatches, ensuring they are actionable.
- Assign a confidence score (0-100) reflecting the reliability of the extracted insights.

Output only the JSON object without additional text.

"""

In [ ]:
import json
import gradio as gr

# Define your model and API keys
llama3_model = 'llama3-70b-8192'
key_1 = userdata.get('ahmed92_groq_key')
key_2 = userdata.get('hamza_groq')
key_3 = userdata.get('tunDC_groq')
api_keys_list = [key_3]


# Updated Wrapper function for Gradio
def analyze_job(job_desc):
    # Call the analysis function
    analysis_text = analyze_job_with_retries(job_desc, system_prompt, llama3_model, api_keys_list)

    # Parse the JSON string into a Python dictionary
    try:
        analysis = json.loads(analysis_text)
    except json.JSONDecodeError as e:
        return f"Error parsing JSON: {e}", "", "", "", "", "", ""

    # Format the output for Gradio
    role_summary = analysis["role_summary"]

    # Format key terms as HTML cards
    key_terms = "".join([
        f"""
        <div class="term-card">
            <div style="font-weight: 600; color: #2b5876;">{term['term']}</div>
            <div style="color: #6c757d; margin-top: 0.25rem;">{term['explanation']}</div>
        </div>
        """
        for term in analysis["key_terms"]
    ])

    must_have_skills = "\n".join(analysis["skill_priorities"]["must_have"])
    nice_to_have_skills = "\n".join(analysis["skill_priorities"]["nice_to_have"])

    # Format screening questions as Q&A pairs
    screening_questions = "".join([
        f"""
        <div class="qa-pair">
            <div class="qa-question">
                <span style="font-size: 1.1em;">❓</span>
                {q['question']}
            </div>
            <div class="qa-answer">
                <span style="font-size: 1.1em;">💡</span>
                {q['example_answer']}
            </div>
        </div>
        """
        for q in analysis["proposed_screening_questions_with_answers"]
    ])

    red_flags = "\n".join(analysis["red_flags"])
    confidence_score = f"**Confidence Score**: {analysis['confidence_score']}%"

    return role_summary, key_terms, must_have_skills, nice_to_have_skills, screening_questions, red_flags, confidence_score

# Gradio Interface
custom_css = """
#jd-container {background: #f8f9fa; padding: 2rem; border-radius: 15px; box-shadow: 0 4px 6px rgba(0,0,0,0.1);}
.header-section {background: linear-gradient(135deg, #2b5876 0%, #4e4376 100%); padding: 2rem; color: white; border-radius: 15px;}
.result-card {background: white; padding: 1.5rem; margin: 1rem 0; border-radius: 10px; box-shadow: 0 2px 4px rgba(0,0,0,0.05);}
.confidence-badge {background: #4CAF50!important; color: white!important; padding: 0.5rem 1rem!important; border-radius: 20px!important;}
.term-card {background: #ffffff; padding: 1rem; margin: 0.5rem 0; border-radius: 8px; border-left: 4px solid #2b5876; box-shadow: 0 2px 4px rgba(0,0,0,0.05); transition: transform 0.2s ease;}
.term-card:hover {transform: translateY(-2px);}
.qa-pair {background: #f8f9fa; padding: 1.25rem; margin: 1rem 0; border-radius: 10px; border: 1px solid #e9ecef;}
.qa-question {color: #2b5876; font-weight: 600; margin-bottom: 0.5rem; display: flex; align-items: center; gap: 0.5rem;}
.qa-answer {color: #6c757d; padding-left: 1.5rem; border-left: 2px solid #dee2e6; margin-left: 1rem; display: flex; gap: 0.5rem;}
.qa-container {padding: 1rem 0;}
"""

with gr.Blocks(css=custom_css, theme=gr.themes.Default(primary_hue="blue", secondary_hue="slate")) as demo:
    with gr.Column(elem_id="main-container"):
        gr.Markdown("""
        <div class="header-section">
            <h1 style="margin: 0; font-weight: 800;">🔍 Job Description Analyzer</h1>
            <p style="margin: 0.5rem 0 0; opacity: 0.9;">AI-powered analysis for better hiring decisions</p>
        </div>
        """)

        with gr.Row():
            with gr.Column(elem_id="jd-container"):
                job_desc = gr.Textbox(label="📋 Paste Job Description",
                                     lines=18,
                                     placeholder="""Example:
• 5+ years experience in full-stack development
• Proficiency in Python and JavaScript
• Familiarity with cloud platforms (AWS preferred)
• Excellent communication skills""",
                                     elem_classes="input-box")

                analyze_button = gr.Button(
                                          value="Analyze Now",  # Button text
                                          variant="primary",   # Primary style
                                          size="lg",           # Large size
                                      )

            with gr.Column():
                with gr.Tab("📌 Key Insights"):
                    role_summary = gr.Textbox(label="📄 Role Summary", lines=4)
                    gr.Markdown("### 🔑 Key Terms & Concepts")
                    key_terms = gr.HTML(label="")  # Changed to HTML component
                    confidence_score = gr.Markdown(label="📊 Confidence Score")

                with gr.Tab("💼 Skills & Requirements"):
                    with gr.Row():
                        must_have_skills = gr.Textbox(label="✅ Must-Have Skills", lines=8,
                                                     elem_classes="result-card")
                        nice_to_have_skills = gr.Textbox(label="✨ Nice-to-Have Skills", lines=8,
                                                        elem_classes="result-card")

                with gr.Tab("❓ Screening Questions"):
                    gr.Markdown("### Suggested Interview Questions")
                    screening_questions = gr.HTML(label="",
                                                 elem_classes="qa-container")  # Changed to HTML component

                with gr.Tab("🚩 Red Flags"):
                    red_flags = gr.Textbox(label="Potential Warning Signs", lines=6,
                                          elem_classes="result-card")

    # Connect the button to the function
    analyze_button.click(
        analyze_job,
        inputs=job_desc,
        outputs=[role_summary, key_terms, must_have_skills, nice_to_have_skills, screening_questions, red_flags, confidence_score]
    )

# Launch the app
demo.launch(debug=True)

Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://3eb3f6c825f993cbd5.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7861 <> https://3eb3f6c825f993cbd5.gradio.live
